# Prerequisites

Run ElasticSearch in Docker
```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

Check if ElasticSearch is running

In [2]:
!curl localhost:9200

{
  "name" : "d9dd33b76d2f",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Pg-hhDDyQG6d1RjQarpknA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


# Import

In [9]:
import json
from elasticsearch import Elasticsearch
from openai import OpenAI
from tqdm.auto import tqdm # to show progress bar

# Implement

In [21]:
es_client = Elasticsearch('http://localhost:9200')
client = OpenAI()

# Create index in ElasticSearch

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
with open('faq_documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

# Merge all faq from 3 coureses into one list
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

100%|██████████| 948/948 [00:04<00:00, 194.99it/s]


In [10]:
query = 'I just disovered the course. Can I still join it?'

In [14]:
def elastic_search(query):
    search_query = {
        "size": 5, # return top 5 results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # question field is 3 times important
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [26]:
result = elastic_search(query)
for r in result:
    print(json.dumps(r, indent=2))

{
  "text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  "section": "General course-related questions",
  "question": "Course - Can I still join the course after the start date?",
  "course": "data-engineering-zoomcamp"
}
{
  "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.",
  "section": "General course-related questions",
  "question": "Course - Can I follow the course after it finishes?",
  "course": "data-engineering-zoomcamp"
}
{
  "text": "You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 

In [16]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
rag(query)

'Yes, you can still join the course even after its start date. There is no need to register, and you are still eligible to submit the homeworks. However, be mindful of the deadlines for turning in the final projects.'

In [23]:
rag("how to run pyspark in docker?")

'The CONTEXT does not provide specific instructions for running PySpark in Docker. You might need to refer to additional resources or documentation for detailed steps on setting up PySpark within a Docker container.'